In [1]:
import torch
import sys,os
sys.path.append(os.pardir)
from bert_seq2seq.tokenizer import Tokenizer, load_chinese_base_vocab
import json
import time
import bert_seq2seq
from bert_seq2seq.utils import load_bert

model = "./model_file/trained_model/summary_roberta_wwm.bin"
model2 = "./model_file/trained_model/summary_roberta_wwm_256.bin"
# pretrian = "./model_file/roberta_wwm.bin"
vocab_text = "./vocab/vocab.txt"
model_name = "roberta"
# writer = SummaryWriter("log")

device = torch.device("cpu")

# 加载字典
word2idx,_ = load_chinese_base_vocab(vocab_path=vocab_text,simplfied=True)
tokenizer = Tokenizer(word2idx)
# 定义模型
bert_model = load_bert(word2idx,model_name=model_name)
bert_model.set_device(device)
bert_model.eval()
bert_model.load_all_params(model_path=model, device=device)

精简后的词表大小为：13584
./model_file/trained_model/summary_roberta_wwm.bin loaded!


In [9]:
bert_model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

In [2]:
model_int8 = torch.quantization.quantize_dynamic(
    bert_model,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8)  


In [3]:
import pandas as pd
df = pd.read_csv("./dataset/train_with_summ.csv")
del df["Unnamed: 0"]
df.head()
text = df["article"][:1]

In [7]:
model_int8.save_all_params("./model_file/trained_model/int8_roberta.pth")